In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!pip install langchain
!pip install torch
!pip install accelerate
!pip install sentence-transformers
!pip install streamlit
!pip install streamlit-chat
!pip install faiss-cpu
!pip install tiktoken
!pip install huggingface-hub
!pip install pypdf
!pip install llama-cpp-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 3.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)


In [ ]:
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 7.9 MB/s eta 0:00:00


In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import PyPDFLoader
import os
import tempfile

In [ ]:
def initialize_session_state():
    session_state = {
        "history": [],
        "generated": ["Hello! Ask me anything about 🤖"],
        "past": ["Hey! 👋"]
    }
    return session_state


In [ ]:
def conversation_chat(query, chain, session_state):
    result = chain({"question": query, "chat_history": session_state["history"]})
    session_state["history"].append((query, result["answer"]))
    return result["answer"]


In [ ]:
def display_chat_history(chain, session_state):
    user_input = input("Enter your question: ")
    if not user_input:
        return

    print(f"Repeating question '{user_input}' 10 times:")

    for _ in range(10):
        output = conversation_chat(user_input, chain, session_state)
        print("Bot:", output)



In [ ]:
from langchain.llms import DeepInfra


In [ ]:
os.environ["DEEPINFRA_API_TOKEN"] = "ohnGw99RAl4ZyJ6zI8gb7RpMGEZ30yHt"

In [ ]:
from langchain.llms import DeepInfra
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

def create_conversational_chain(vector_store):
    local_llm = DeepInfra(model_id="mistralai/Mistral-7B-Instruct-v0.1",
        #streaming=True,
        #temperature=0.75,
        #top_p=1,
        #verbose=True,
        #n_ctx=4096,
    )

    local_llm.model_kwargs = {
        "streaming":True,
        "temperature": 0.75,
        "top_p":1,
        "verbose":True,
        "n_ctx" : 4096,
    }

    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    chain = ConversationalRetrievalChain.from_llm(
        llm=local_llm,
        chain_type="stuff",
        retriever=vector_store.as_retriever(search_kwargs={"k": 2}),
        memory=memory
    )
    return chain

In [ ]:
def main():
    session_state=initialize_session_state()
    print("ChatBot using Mistral-7B-Instruct LLM :books:")

    uploaded_files = ["/content/University2.pdf"]

    if uploaded_files:
        text = []
        for file_path in uploaded_files:
            with open(file_path, "rb") as f:
                file_contents = f.read()
            file_extension = os.path.splitext(file_path)[1]
            with tempfile.NamedTemporaryFile(delete=False) as temp_file:
                temp_file.write(file_contents)
                temp_file_path = temp_file.name
                print("loading: ", file_path)
            loader = None
            if file_extension == ".pdf":
                loader = PyPDFLoader(temp_file_path)

            if loader:
                text.extend(loader.load())
                os.remove(temp_file_path)

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=10000, chunk_overlap=20
        )
        text_chunks = text_splitter.split_documents(text)

        print("chunks:\n", text_chunks)

        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2",
            model_kwargs={"device": "cpu"},
        )

        print("embeddings:\n", embeddings)

        vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

        chain = create_conversational_chain(vector_store)

        display_chat_history(chain,session_state)


if __name__ == "__main__":
    main()

ChatBot using Mistral-7B-Instruct LLM :books:
loading:  /content/University2.pdf
chunks:
 [Document(page_content='The Faculty of Information Technology is the youngest of the three faculties of the  University \nof Moratuwa  and is the first ever faculty of its kind in the national university system of Sri \nLanka. The Faculty was established in June 2001 to cater to the growing need of Information \nTechnology professionals in the country. Presently, the Faculty conducts two int ernal degree \nprogrammes, namely,  B.Sc. (Hons.) in Information Technology  and B.Sc. (Hons.) in Information \nTechnology & Management , and the  Bachelor of Information Technology , the external degree \nprogramme.  \n \nOne of the main objectives of setting up the faculty is to increase the number of IT \nprofessionals in Sri Lanka by at least another 500. Some of the other objectives defined during \nthe establishment of the faculty are the introduction of postgraduate progr ammes, carrying \nout research 

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


embeddings:
 client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='sentence-transformers/all-MiniLM-L6-v2' cache_folder=None model_kwargs={'device': 'cpu'} encode_kwargs={} multi_process=False show_progress=False
Enter your question: what are the three departments?
Repeating question 'what are the three departments?' 10 times:
Bot:  The three departments are Department of Information Technology, Department of Computational Mathematics, and Department of Interdisciplinary Studies.
Bot:   The Faculty of Information Technology in the University of Moratuwa  has three 
departments of study, n